In [1]:
path = 'C:/Users/isaac/Dropbox/Statistics/P25/information_lawyer_quality'

In [2]:
import pandas as pd
dem_iniciales = pd.read_excel('C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality/Raw/iniciales_dem/dulce.xlsx')
dem_iniciales.head()

,EXPEDIENTE,LUGAURL,ESTATUS,FECHADETERMINO,MODODETERMINO,UBICACIÓN,CANTIDAD,CANTIDADCOBRADA1SI0NO2NO,COINCIDEACTOR1SI0NO2NOE,OBSERVACIONES,OBSERVACIONESDEIDACTOR,TRATAMIENTO,OBSERVACIONESGENERALESDELEXPE,auxiliar,junta,expediente,anio,id_actor
0,5_190_2019,https://drive.google.com/file/d/1pv1Fwnbca3reZ...,CONTINUA,NaN,NaN,NaN,NaN,NaN,1,"CONCILIACION, DEMANDA Y EXCEPCIONES",TRATAMIENTO POR 6 DEMANDA POR 6 (1578_1 LETICI...,1B,NaN,NaN,NaN,NaN,NaN,1583_1
1,5_190_2019,https://drive.google.com/file/d/1pv1Fwnbca3reZ...,CONTINUA,NaN,NaN,NaN,NaN,NaN,1,"CONCILIACION, DEMANDA Y EXCEPCIONES",TRATAMIENTO POR 6 DEMANDA POR 6 (1578_1 LETICI...,1B,NaN,NaN,NaN,NaN,NaN,1582_1
2,7_2065_2018,https://drive.google.com/file/d/1mZ_mpnF_se_Ul...,TERMINO,04mar2019,DESISTIMIENTO,NaN,NaN,NaN,1,NaN,TRATAMIENTO POR 4 DEMANDA POR 4 (1154_1 JOSE ...,1A,NaN,NaN,NaN,NaN,NaN,1157_1
3,5_190_2019,https://drive.google.com/file/d/1pv1Fwnbca3reZ...,CONTINUA,NaN,NaN,NaN,NaN,NaN,1,"CONCILIACION, DEMANDA Y EXCEPCIONES",TRATAMIENTO POR 6 DEMANDA POR 6 (1578_1 LETICI...,1B,NaN,NaN,NaN,NaN,NaN,1581_1
4,8_306_2019,https://drive.google.com/file/d/1ZOyPHBjyd_qiE...,CONTINUA,NaN,NaN,NaN,NaN,NaN,1,"CONCILIACION, DEMANDA Y EXCEPCIONES",TRATAMIENTO POR 4 DEMANDA POR 8 (1659_1AYLEEN ...,1A,NaN,NaN,NaN,NaN,NaN,1662_1


In [3]:
# def url_to_id(url):
#     x = url.split("/")
#     return x[5]

# import sys
# !{sys.executable} -m pip install gdown
# import gdown 

# # Download casefiles
# for index, row in dem_iniciales.iterrows():
#     id = url_to_id(row['LUGAURL'])
#     url = 'https://drive.google.com/uc?id=' + id
#     output = path + '/Raw/escaneos/' + row['EXPEDIENTE'] + '.pdf'
#     gdown.download(url, output, quiet=False)      

In [11]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import gc

In [12]:
# Remove noise
def remove_noise(img):
    return cv2.medianBlur(img, 5)

# Thresholding
def threshold(img):
    # return cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# skew correction
def deskew(img):
    coords = np.column_stack(np.where(img > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = img.shape[:2]
    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(
        img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated



In [ ]:
directory = os.fsencode(path + '/Raw/escaneos/')

# Create corpus
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".pdf") :
        print(filename)

        gc.collect()
        # Path of the pdf
        PDF_file = path + '/Raw/escaneos/' + filename

        '''
        Part #1 : Converting PDF to images
        '''

        # Store all the pages of the PDF in a variable
        pages = convert_from_path(PDF_file, 500, poppler_path=r'C:\Program Files\poppler-0.68.0_x86\bin')

        # Counter to store images of each page of PDF to image
        image_counter = 1

        # Iterate through all the pages stored above
        for page in pages:
            # Declaring filename for each page of PDF as JPG
            # For each page, filename will be:
            # PDF page n -> page_n.jpg
            imgname = path + "./Raw/escaneos/img/page_"+str(image_counter)+".jpg"

            # Save the image of the page in system
            page.save(imgname, 'JPEG')

            # Increment the counter to update filename
            image_counter = image_counter + 1
        '''
        Part #2 - Recognizing text from the images using OCR
        '''
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

        # Variable to get count of total number of pages
        filelimit = image_counter-1

        # Creating a text file to write the output
        outfile = path + "./Raw/escaneos/text/" + os.path.splitext(filename)[0] + ".txt"

        # Open the file in append mode so that 
        # All contents of all images are added to the same file
        f = open(outfile, "a")
        
        # Iterate from 1 to total number of pages
        for i in range(1, filelimit + 1):
            print("img #:", i,"/",filelimit)

            # Set filename to recognize text from
            # Again, these files will be:
            # page_n.jpg
            imgname = path + "./Raw/escaneos/img/page_"+str(i)+".jpg"

            img = cv2.imread(imgname, cv2.IMREAD_GRAYSCALE)    
            #img = deskew(threshold(remove_noise(img)))
            # Recognize the text as string in image using pytesserct
            # 0    Legacy engine only.
            # 1    Neural nets LSTM engine only.
            # 2    Legacy + LSTM engines.
            # 3    Default, based on what is available.
            text = str(((pytesseract.image_to_string(img,lang='spa',
                                                     config = r'--oem 3 --psm 6'))))

            # The recognized text is stored in variable text
            # Any string processing may be applied on text
            # Here, basic formatting has been done:
            # In many PDFs, at line ending, if a word can't
            # be written fully, a 'hyphen' is added.
            text = text.replace('-\n', '')    

            # Finally, write the processed text to the file.
            f.write(text)

        # Close the file after writing all the text.
        f.close()

11_1420_2018.pdf


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\isaac\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\isaac\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\isaac\anaconda3\lib\subprocess.py", line 1370, in _readerthread
    buffer.append(fh.read())
MemoryError


**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...